In [1]:
import os
import json
import boto3
import sagemaker
from time import gmtime, strftime

from sagemaker.xgboost import XGBoost
from sagemaker.model import Model
from sagemaker.model_metrics import MetricsSource, ModelMetrics
from sagemaker.model_card.model_card import (
    ModelCard,
    TrainingDetails,
    TrainingJobDetails,
    ModelOverview,
    IntendedUses,
    AdditionalInformation,
    Metric,
    MetricTypeEnum,
    HyperParameter,
    Environment,
    TrainingMetric,
    ObjectiveFunction,
    Function,
    ObjectiveFunctionEnum,
    FacetEnum,
    RiskRatingEnum,
    ModelCardStatusEnum,
    ModelPackage as CardModelPackage,
    BusinessDetails,
    EvaluationJob,
    MetricGroup

)
from botocore.exceptions import ClientError

import mlflow as mlf
import sagemaker_mlflow  # activate plugin

# -------------------------------------------------
# Config พื้นฐาน
# -------------------------------------------------
sess = sagemaker.Session()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
sm = boto3.client("sagemaker", region_name=region)
sts = boto3.client("sts", region_name=region)
account_id = sts.get_caller_identity()["Account"]
s3 = boto3.client("s3", region_name=region)

print("Region:", region)
print("Role:", role)



sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


/home/sagemaker-user/.conda/envs/data_science_on_aws/lib/python3.12/site-packages/mlflow/utils/requirements_utils.py:20: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources  # noqa: TID251


Region: us-east-1
Role: arn:aws:iam::423623839320:role/service-role/SageMaker-ExecutionRole-20250705T232334


In [2]:
# โหลดค่าจาก store
%store -r training_job_name
%store -r eval_output_s3
%store -r processed_train_data_s3_uri   

print("Training job name:", training_job_name)
print("Eval output S3 prefix:", eval_output_s3)
print("Processed train data S3 URI:", processed_train_data_s3_uri)


Training job name: sagemaker-xgboost-2025-12-06-13-01-48-299
Eval output S3 prefix: s3://sagemaker-us-east-1-423623839320/retail-demand/evaluation-2025-12-03-10-49-06/
Processed train data S3 URI: s3://sagemaker-us-east-1-423623839320/sagemaker-scikit-learn-2025-12-03-07-46-41-610/output/retail-train


In [3]:

# # ตรงกับที่ใช้ตอน training
# EXPERIMENT_NAME = "forcasting_demand_product"
# MLFLOW_TRACKING_SERVER_ARN = (
#     "arn:aws:sagemaker:us-east-1:423623839320:mlflow-tracking-server/tracking-server-demo"
# )

# # ตั้งค่า MLflow
# mlf.set_tracking_uri(MLFLOW_TRACKING_SERVER_ARN)
# mlf.set_experiment(EXPERIMENT_NAME)


In [4]:
# -------------------------------------------------
# 1. Attach estimator + เตรียมค่า model
# -------------------------------------------------
xgb_estimator = XGBoost.attach(training_job_name=training_job_name)
model_data = xgb_estimator.model_data
print("Model data S3:", model_data)

# image ที่จะใช้ infer (ให้ตรงกับ training)
XGBOOST_IMAGE_URI = xgb_estimator.image_uri
print("XGBoost image URI:", XGBOOST_IMAGE_URI)



2025-12-06 13:07:13 Starting - Preparing the instances for training
2025-12-06 13:07:13 Downloading - Downloading the training image
2025-12-06 13:07:13 Training - Training image download completed. Training in progress.
2025-12-06 13:07:13 Uploading - Uploading generated training model
2025-12-06 13:07:13 Completed - Training job completed/miniconda3/lib/python3.9/site-packages/sagemaker_containers/_server.py:22: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
[2025-12-06 13:03:47.794 ip-10-2-77-224.ec2.internal:7 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2025-12-06 13:03:47.863 ip-10-2-77-224.ec2.internal:7 INFO profiler_config_parser.py:111] User has disabled profiler.
[2025-12-06:13:03:48:INFO] Imported framework sagemaker_xgboost_container.training
[2025-

In [5]:
suffix = strftime("%Y%m%d-%H%M%S", gmtime())
model_card_name = f"retail-demand-xgb-card-{suffix}"
model_package_group_name = "retail-demand-model-group"
model_approval_status = "PendingManualApproval"  # คุณจะ Manual approve ในภายหลัง


# Re-format evaluate file to be registered

In [6]:
s3_eval_json = os.path.join(eval_output_s3 + "evaluation_summary.json")
local_eval_json = "data/evaluation_summary_for_registry.json"

# ----------------------------------------
# 1) โหลด evaluation_summary.json ที่ evaluate.py เขียนไว้
# ----------------------------------------
!mkdir -p data 
!aws s3 cp $s3_eval_json $local_eval_json

with open(local_eval_json, "r") as f:
    summary = json.load(f)

rmse = float(summary["metrics"]["rmse"])
mae  = float(summary["metrics"]["mae"])
r2   = float(summary["metrics"]["r2"])


bias = summary.get("bias_metrics", {})
dpl  = float(bias.get("DPL_labels", float("nan")))
dppl = float(bias.get("DPPL_predicted", float("nan")))
ad   = float(bias.get("AD_accuracy_diff", float("nan")))

eval_timestamp_utc = summary.get("eval_timestamp_utc")


print("Loaded metrics:", rmse, mae, r2)
print("Loaded bias", dpl, dppl, ad)
print("Eval timestamp:", eval_timestamp_utc)

# ----------------------------------------
# 2) สร้าง JSON ใน format ที่ SageMaker ชอบ
# ----------------------------------------

quality_json = {
    "regression_metrics": {
        "rmse": {"value": rmse, "standard_deviation": "NaN"},
        "mae":  {"value": mae,  "standard_deviation": "NaN"},
        "r2":   {"value": r2,   "standard_deviation": "NaN"},
        "dpl": {"value": dpl, "standard_deviation": "NaN"},
        "dppl": {"value": dppl, "standard_deviation": "NaN"},
        "ad": {"value": ad, "standard_deviation": "NaN"},
    }
}


local_quality_json = "data/evaluation_regression_metrics.json"
with open(local_quality_json, "w") as f:
    json.dump(quality_json, f)
# ----------------------------------------
# 3) อัปโหลดขึ้น S3 ใน prefix เดิม
# ----------------------------------------
model_quality_s3_uri = os.path.join(eval_output_s3 + "evaluation_regression_metrics.json")
!aws s3 cp local_quality_json model_quality_s3_uri

print("Model quality JSON S3 URI:", model_quality_s3_uri)

%store model_quality_s3_uri

download: s3://sagemaker-us-east-1-423623839320/retail-demand/evaluation-2025-12-03-10-49-06/evaluation_summary.json to data/evaluation_summary_for_registry.json
Loaded metrics: 21.68276023864746 11.380695343017578 0.7065292051955987
Loaded bias -0.625 -0.375 0.25
Eval timestamp: 2025-12-03T10:51:32.309311Z

usage: aws s3 cp <LocalPath> <S3Uri> or <S3Uri> <LocalPath> or <S3Uri> <S3Uri>
Error: Invalid argument type
Model quality JSON S3 URI: s3://sagemaker-us-east-1-423623839320/retail-demand/evaluation-2025-12-03-10-49-06/evaluation_regression_metrics.json
Stored 'model_quality_s3_uri' (str)


# Model

In [7]:
# -------------------------------------------------
# 2. ModelMetrics : ชี้ไปที่ไฟล์ evaluation บน S3
# -------------------------------------------------
# เรารู้ว่า evaluate.py เขียน evaluation_summary.json และแปลง format ใน evaluation_regression_metrics

model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri=model_quality_s3_uri,
        content_type="application/json",   # 👈 เปลี่ยนเป็น JSON
    )
)

print("Model metrics stats:", model_quality_s3_uri)



Model metrics stats: s3://sagemaker-us-east-1-423623839320/retail-demand/evaluation-2025-12-03-10-49-06/evaluation_regression_metrics.json


In [8]:
# -------------------------------------------------
# 3. สร้าง Model object (ผูก model_data + image + role)
# -------------------------------------------------
model = Model(
    image_uri=XGBOOST_IMAGE_URI,
    model_data=model_data,
    role=role,
    name=xgb_estimator.latest_training_job.name,
    sagemaker_session=sess,
)

print("Model name to register:", xgb_estimator.latest_training_job.name)


Model name to register: sagemaker-xgboost-2025-12-06-13-01-48-299


In [9]:
# -------------------------------------------------
# 4.1 Objective function (ใช้ RMSE บน validation/test)
# -------------------------------------------------
objective_function = ObjectiveFunction(
    function=Function(
        function=ObjectiveFunctionEnum.MINIMIZE,
        facet=FacetEnum.RMSE,
        condition="Minimize RMSE on validation/test set",
    ),
    notes="Objective is to minimize RMSE of units_sold on validation/test data.",
)
# -------------------------------------------------
# 4.2 Training environment (Environment)
# -------------------------------------------------
training_env = Environment(
    container_image=[XGBOOST_IMAGE_URI]  # image URI เดียวสำหรับ training/inference
)

# -------------------------------------------------
# 4.3 Training datasets (S3 URIs ของ train/val/test หลัง preprocess)
# -------------------------------------------------
try:
    %store -r processed_train_data_s3_uri
    %store -r processed_validation_data_s3_uri
    %store -r processed_test_data_s3_uri
    training_datasets = [
        processed_train_data_s3_uri,
        processed_validation_data_s3_uri,
        processed_test_data_s3_uri,
    ]
except Exception:
    # fallback ถ้าไม่มี ก็อย่างน้อย log eval_output_s3 เป็น dataset
    training_datasets = [eval_output_s3]
# -------------------------------------------------
# 4.4 สร้าง hyperparameters  
# -------------------------------------------------
hyperparams_dict = xgb_estimator.hyperparameters()  # dict[name] = value
hyperparams_list = [
    HyperParameter(name=str(k), value=str(v)) for k, v in hyperparams_dict.items()
]

# -------------------------------------------------
# 4.5 Training metrics (ใช้ metrics จาก evaluation เป็น user-provided training metrics)
# -------------------------------------------------
training_metrics_list = [
    TrainingMetric(
        name="test_rmse",
        value=rmse,
        notes="RMSE on held-out test set",
    ),
    TrainingMetric(
        name="test_mae",
        value=mae,
        notes="MAE on held-out test set",
    ),
    TrainingMetric(
        name="test_r2",
        value=r2,
        notes="R-squared on held-out test set",
    ),
]


# -------------------------------------------------
# 4.6 Evaluation metrics (ใช้ metrics จาก evaluation)
# -------------------------------------------------

regression_metric_group = MetricGroup(
    name="Regression metrics (test set)",
    metric_data=[
        Metric(
            name="rmse",
            type=MetricTypeEnum.NUMBER,
            value=rmse,
            notes="RMSE on held-out test set",
        ),
        Metric(
            name="mae",
            type=MetricTypeEnum.NUMBER,
            value=mae,
            notes="MAE on held-out test set",
        ),
        Metric(
            name="r2",
            type=MetricTypeEnum.NUMBER,
            value=r2,
            notes="R-squared on held-out test set",
        ),
    ],
)

bias_metric_group = MetricGroup(
    name="Bias metrics (weekday vs weekend)",
    metric_data=[
        Metric(
            name="DPL_labels",
            type=MetricTypeEnum.NUMBER,
            value=dpl,
            notes="Difference in positive label rate (weekday - weekend)",
        ),
        Metric(
            name="DPPL_predicted",
            type=MetricTypeEnum.NUMBER,
            value=dppl,
            notes="Difference in positive predicted rate (weekday - weekend)",
        ),
        Metric(
            name="AD_accuracy_diff",
            type=MetricTypeEnum.NUMBER,
            value=ad,
            notes="Accuracy difference (weekday - weekend)",
        ),
    ],
)

# ถ้าคุณมี S3 ช่อง test set:
try:
    %store -r processed_test_data_s3_uri
    eval_datasets = [processed_test_data_s3_uri]
except Exception:
    eval_datasets = [eval_output_s3]  # fallback เป็น prefix ของ evaluation

eval_job_name = f"retail-demand-xgb-eval-{strftime('%Y%m%d-%H%M%S', gmtime())}"

evaluation_job = EvaluationJob(
    name=eval_job_name,
    evaluation_observation=(
        "Evaluation on held-out test set. Metrics and bias metrics derived "
        "from evaluation_summary.json."
    ),
    datasets=eval_datasets,
    metric_groups=[regression_metric_group, bias_metric_group],
)

evaluation_details = [evaluation_job]



# -------------------------------------------------
# 4.7 TrainingJobDetails (รวม datasets, env, metrics, hyperparams)
# -------------------------------------------------
training_job_arn = f"arn:aws:sagemaker:{region}:{account_id}:training-job/{training_job_name}"

training_job_details = TrainingJobDetails(
    training_arn=training_job_arn,
    training_datasets=training_datasets,           # list ของ S3 URIs
    training_environment=training_env,
    # training_metrics = metrics ที่ SageMaker เก็บให้โดยอัตโนมัติ (optional)
    user_provided_training_metrics=training_metrics_list,
    hyper_parameters=hyperparams_list,             # hyperparameters ที่ใช้จริง
)

# -------------------------------------------------
# 4.8 TrainingDetails (ผูก objective + training_job_details)
# -------------------------------------------------
training_details = TrainingDetails(
    objective_function=objective_function,
    training_observations="XGBoost regression to forecast units_sold (bottled water) using calendar, weather, and promotion features.",
    training_job_details=training_job_details,
)

# -------------------------------------------------
# 4.9 IntendedUses (purpose, uses, risk, etc.)
# -------------------------------------------------
intended_uses = IntendedUses(
    purpose_of_model=(
        "Forecast daily bottled water demand per store to support inventory "
        "planning and anomaly/fraud detection on unusual slips."
    ),
    intended_uses=(
        "Use for short-term demand forecasting, safety stock calculation, and "
        "flagging abnormal demand patterns for investigation."
    ),
    factors_affecting_model_efficiency=(
        "Seasonality (weekends, holidays), weather (max_temp_c, rainfall_mm), "
        "and promotions (is_promo, discount_pct) strongly affect performance. "
        "Large price policy changes or new product mixes may require retraining."
    ),
    risk_rating=RiskRatingEnum.MEDIUM,
    explanations_for_risk_rating=(
        "Model impacts financial and operational decisions (stock, cash-flow), "
        "but does not directly approve/deny customers or make safety-critical decisions."
    ),
)

# -------------------------------------------------
# 4.10 AdditionalInformation – ใช้สำหรับ Audit > Documentation / Ethical
# -------------------------------------------------

additional_info = AdditionalInformation(
    ethical_considerations=(
        "Avoid using this model for decisions that directly impact individuals "
        "such as creditworthiness or hiring. Monitor for bias in store segments "
        "if used for resource allocation across regions."
    ),
    caveats_and_recommendations=(
        "Model trained on 2024 data only. If store network, pricing, or "
        "consumer behavior changes significantly, retrain and re-evaluate metrics."
    ),
)
# -------------------------------------------------
# 4.11 BusinessDetails
# -------------------------------------------------

business_details = BusinessDetails(
    business_problem=(
        "Reduce stockouts and overstock of bottled water across retail stores "
        "by forecasting daily demand more accurately."
    ),
    business_stakeholders=(
        "Data science team (model development and monitoring), "
        "supply chain & inventory planning team, store operations, and finance."
    ),
    line_of_business="Retail grocery / convenience stores – beverages category",
)

# -------------------------------------------------
# 4.12 ModelOverview 
# -------------------------------------------------
model_overview = ModelOverview(
    model_name=xgb_estimator.latest_training_job.name,
    model_description="XGBoost regression model for retail bottled water demand forecasting.",
    model_creator = "Smith Company",
    model_owner = "Smith Brave",
    problem_type="Regression",
    algorithm_type="XGBoost",
    model_artifact=[model_data],
    inference_environment=Environment(container_image=[XGBOOST_IMAGE_URI]),
)






In [10]:
# -------------------------------------------------
# 5. Register model เข้าสู่ Model Registry
# -------------------------------------------------
print("Registering model into Model Package Group:", model_package_group_name)

model_package = model.register(
    content_types=["text/csv"],
    response_types=["application/json"],
    inference_instances=["ml.m5.large", "ml.m5.xlarge", "ml.m5.2xlarge"],
    transform_instances=["ml.m5.large", "ml.m5.xlarge"],
    domain="MACHINE_LEARNING",
    task="REGRESSION",
    model_package_group_name=model_package_group_name,
    model_metrics=model_metrics,
    approval_status=model_approval_status,  # "PendingManualApproval"
)

model_package_arn = model_package.model_package_arn
print("Registered Model Package ARN:", model_package_arn)

mp_details = CardModelPackage.from_model_package_arn(
    model_package_arn=model_package_arn
)

# เผื่อเอาไปใช้ใน cell ถัดไป
%store model_package_arn

Registering model into Model Package Group: retail-demand-model-group


Registered Model Package ARN: arn:aws:sagemaker:us-east-1:423623839320:model-package/retail-demand-model-group/6
Stored 'model_package_arn' (str)


In [11]:
# -------------------------------------------------
# 6 ModelCard (รวม overview + intended uses + training details)
# -------------------------------------------------
from sagemaker.model_card.model_card import ModelCardStatusEnum

model_card = ModelCard(
    name=model_card_name,
    status=ModelCardStatusEnum.DRAFT,
    model_overview=model_overview,
    intended_uses=intended_uses,
    training_details=training_details,
    additional_information=additional_info,
    business_details=business_details,
    evaluation_details=evaluation_details,
    # model_package_details=mp_details,    # 👈 ตรงนี้สำคัญมาก
    sagemaker_session=sess,
)



print("Creating model card:", model_card_name)
model_card.create()
print("Model card created.")




Creating model card: retail-demand-xgb-card-20251206-130950


Model card created.


In [13]:
# from sagemaker.model_card.model_card import ModelCard

# card_name = model_card_name # ดูจาก console หรือที่คุณเก็บไว้
# model_card = ModelCard.load(card_name, sagemaker_session=sess)

# # เซ็ตฟิลด์เพิ่ม
# model_card.model_overview = model_overview          # มี creator / owner
# model_card.intended_uses = intended_uses            # มี risk rating + usage
# model_card.business_details = business_details      # business/stakeholders
# model_card.additional_information = additional_info # documentation/ethical
# model_card.training_details = training_details      # ถ้ายังไม่เคยใส่

# # update() จะสร้าง version ใหม่ของ model card (เวอร์ชันเดิมเป็น immutable)
# model_card.update(status=model_card.status)
# print("Updated model card:", card_name)


Updated model card: retail-demand-xgb-card-20251205-140521


Stored 'model_package_arn' (str)


In [ ]:
# -------------------------------------------------
# 6. Log เข้า MLflow
# -------------------------------------------------
with mlf.start_run(
    run_name=f"register-model-{suffix}",
    description="Register XGBoost model into SageMaker Model Registry",
):
    mlf.log_params(
        {
            "training_job_name": training_job_name,
            "model_name": model_name,
            "model_package_group_name": model_package_group_name,
            "model_package_arn": model_package_arn,
            "model_approval_status": model_approval_status,
            "eval_summary_s3_uri": eval_summary_s3_uri,
            "shap_s3_uri": shap_s3_uri,
        }
    )

# เผื่อเอาไปใช้ใน cell ถัดไป
%store model_package_arn
